In [1]:
# from dotenv import load_dotenv
# load_dotenv()
# import os
# os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [2]:
#Creating wiki tool -->

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
#Creating arxiv tool -->

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [4]:
# Creating retriever -->

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents=splitter.split_documents(docs)
vectordb=FAISS.from_documents(documents,hf_embeddings)
retriever=vectordb.as_retriever()

C:\Users\DELL\AppData\Local\Temp\ipykernel_16520\629336791.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
d:\ML\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Creating Retriever tool -->

from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search", "Search for information about langsmith. For any question about langsmith, you must use this tool")

In [7]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
# from langchain.llms import HuggingFacePipeline
# from langchain.agents import initialize_agent, AgentType

# model_name = "google/flan-t5-small" 
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
# llm = HuggingFacePipeline(pipeline=pipe)

In [8]:
from langchain_ollama.llms import OllamaLLM
llm=OllamaLLM(model="gemma3:4b", temperature=0.1,  top_p=0.9)

In [9]:
from langchain import hub
prompt=hub.pull("hwchase17/react")
tools=[wiki_tool,retriever_tool,arxiv_tool]
# tools=[retriever_tool]

d:\ML\venv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
#Agent executor -->
from langchain.agents import create_react_agent, AgentExecutor
agent=create_react_agent(llm, tools, prompt)
agent_executor=AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)

In [11]:
result=agent_executor.invoke({"input": "What is highway hypnosis?"})
print(result)



> Entering new AgentExecutor chain...
I need to understand what highway hypnosis is. I can use the wikipedia tool to find information about this topic.
Action: wikipedia
Action Input: "highway hypnosis"Page: Highway hypnosis
Summary: Highway hypnosis, also known as white line fever, is an altered mental state in which an automobile driver can drive lengthy distances and respond adequately to externaI now know the final answer
Final Answer: Page: Highway hypnosis
Summary: Highway hypnosis, also known as white line fever, is an altered mental state in which an automobile driver can drive lengthy distances and respond adequately to externa

> Finished chain.
{'input': 'What is highway hypnosis?', 'output': 'Page: Highway hypnosis\nSummary: Highway hypnosis, also known as white line fever, is an altered mental state in which an automobile driver can drive lengthy distances and respond adequately to externa'}
